# Prediciton from NFL combine data

Shun Iwata
Course Project, UC Irvine, Math 10, S22

## Introduction

In this project, I am investigating which classifier is the best fit to predict player's position from the NFL combine data. Mainly using `LinearRegression` and `DecisionTreeClassifier`, I modified (meaning making the categories more braod) the position column and applying columns to seek for better predicition. By the end, we will see if we are able to distingush player's position from the data measured in combine. 

## Main portion of the project

### Import data

I fisrt imported two datasets because the sample in the combine 2020 data was not enough (I personally thought) to make a good prediction. 
Thus, I also imported the dataset contains all the combine data from 2000 up till 2018 as `df2`.

In [ ]:
import pandas as pd
df=pd.read_csv("NFL_2020_Combine.csv")
df2=pd.read_csv("combine.csv")
df.head()

,Unnamed: 0,Player,Pos,Ht,Wt,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle
0,0,Trey Adams,OL,203,318,5.60,24.5,NaN,92.0,NaN,NaN
1,1,Hakeem Adeniji,OL,193,302,5.17,34.0,26.0,115.0,NaN,NaN
2,2,McTelvin Agim,DL,190,309,4.98,NaN,27.0,NaN,NaN,NaN
3,3,Salvon Ahmed,RB,180,197,4.62,34.5,NaN,120.0,NaN,NaN
4,4,Brandon Aiyuk,WR,183,205,4.50,40.0,11.0,128.0,NaN,NaN


In [ ]:
df2.head()

,Player,Pos,Ht,Wt,Forty,Vertical,BenchReps,BroadJump,Cone,Shuttle,Year,Pfr_ID,AV,Team,Round,Pick
0,John Abraham,OLB,76,252,4.55,NaN,NaN,NaN,NaN,NaN,2000,AbraJo00,26.0,New York Jets,1.0,13.0
1,Shaun Alexander,RB,72,218,4.58,NaN,NaN,NaN,NaN,NaN,2000,AlexSh00,26.0,Seattle Seahawks,1.0,19.0
2,Darnell Alford,OT,76,334,5.56,25.0,23.0,94.0,8.48,4.98,2000,AlfoDa20,0.0,Kansas City Chiefs,6.0,188.0
3,Kyle Allamon,TE,74,253,4.97,29.0,NaN,104.0,7.29,4.49,2000,NaN,0.0,NaN,NaN,NaN
4,Rashard Anderson,CB,74,206,4.55,34.0,NaN,123.0,7.18,4.15,2000,AndeRa21,6.0,Carolina Panthers,1.0,23.0


As I look the data, the `Ht` (height) of `df2` was in inches, so I multiplied the `Ht` column by 2.54 because 1 inches equal to 2.54cm.

In [ ]:
df2["Ht"]*=2.54
df2.Ht

0       193.04
1       182.88
2       193.04
3       187.96
4       187.96
         ...  
6213    187.96
6214    190.50
6215    185.42
6216    185.42
6217    193.04
Name: Ht, Length: 6218, dtype: float64

By using `.info()`, I make sure all the numerical data are in either integer or float.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337 entries, 0 to 336
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  337 non-null    int64  
 1   Player      337 non-null    object 
 2   Pos         337 non-null    object 
 3   Ht          337 non-null    int64  
 4   Wt          337 non-null    int64  
 5   40yd        267 non-null    float64
 6   Vertical    250 non-null    float64
 7   Bench       216 non-null    float64
 8   Broad Jump  239 non-null    float64
 9   3Cone       154 non-null    float64
 10  Shuttle     160 non-null    float64
dtypes: float64(6), int64(3), object(2)
memory usage: 29.1+ KB


In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6218 entries, 0 to 6217
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Player     6218 non-null   object 
 1   Pos        6218 non-null   object 
 2   Ht         6218 non-null   float64
 3   Wt         6218 non-null   int64  
 4   Forty      6046 non-null   float64
 5   Vertical   4796 non-null   float64
 6   BenchReps  4212 non-null   float64
 7   BroadJump  4754 non-null   float64
 8   Cone       3993 non-null   float64
 9   Shuttle    4063 non-null   float64
 10  Year       6218 non-null   int64  
 11  Pfr_ID     4895 non-null   object 
 12  AV         6218 non-null   float64
 13  Team       3738 non-null   object 
 14  Round      3738 non-null   float64
 15  Pick       3738 non-null   float64
dtypes: float64(10), int64(2), object(4)
memory usage: 777.4+ KB


### Concatenate the datasets

By looking at the `.info()` labels, I concatenate the two dataframes into one to make the later process easier.

In [ ]:
cols=df.columns[2:]
df2.rename({"Forty":cols[3],"BenchReps":cols[5],"BroadJump":cols[-3],"Cone":cols[-2]}, axis=1, inplace=True)
df_sub=df[cols]
df2_sub=df2[cols]

In [ ]:
df2_sub.shape

(6218, 9)

In [ ]:
df_sub.shape

(337, 9)

In [ ]:
df_com=pd.concat([df_sub,df2_sub])
df_com.head()

,Pos,Ht,Wt,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle
0,OL,203.0,318,5.60,24.5,NaN,92.0,NaN,NaN
1,OL,193.0,302,5.17,34.0,26.0,115.0,NaN,NaN
2,DL,190.0,309,4.98,NaN,27.0,NaN,NaN,NaN
3,RB,180.0,197,4.62,34.5,NaN,120.0,NaN,NaN
4,WR,183.0,205,4.50,40.0,11.0,128.0,NaN,NaN


I also realized that `Pos` column in `df2` has more sepcified positions comapred to the `df`, thus I create a fuction that synthesize the `Pos` column in `df_com`.

In [ ]:
df["Pos"].unique()

array(['OL', 'DL', 'RB', 'WR', 'CB', 'TE', 'LB', 'K', 'S', 'QB', 'P',
       'LS'], dtype=object)

In [ ]:
df2["Pos"].unique()

array(['OLB', 'RB', 'OT', 'TE', 'CB', 'K', 'P', 'FS', 'OG', 'ILB', 'DE',
       'SS', 'DT', 'QB', 'WR', 'C', 'FB', 'LS', 'NT', 'EDGE', 'S', 'G',
       'LB', 'DB', 'OL'], dtype=object)

In [ ]:
def position(x):
    if x in ["OLB","ILB"]:
        return "LB"
    elif x in ['FS','SS']:
        return "S"
    elif x in ['OT','OG','G','C']:
       return "OL"
    elif x in ['DE','DT','EDGE','NT']:
        return "DL"
    elif x in ["FB"]:
        return "RB"
    else:
        return x

In [ ]:
df_com["Pos"]=df_com["Pos"].map(position)
df_com["Pos"].unique()

array(['OL', 'DL', 'RB', 'WR', 'CB', 'TE', 'LB', 'K', 'S', 'QB', 'P',
       'LS', 'DB'], dtype=object)

In [ ]:
df_com=df_com.dropna()

### Prediction using LogisticRegression

First, specify the numerical data columns that I am going to use for prediction creating `cols`.

In [ ]:
cols=df_com.columns[1:]
cols

Index(['Ht', 'Wt', '40yd', 'Vertical', 'Bench', 'Broad Jump', '3Cone',
       'Shuttle'],
      dtype='object')

Then, using `train_test_split`, create train set and test set.

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df_com[cols],df_com["Pos"],train_size=0.8)

In [ ]:
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression()
clf.fit(X_train,y_train)
clf.score(X_train,y_train)

/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.5967336683417085

It is hard to conclude that this is a good prediction score.

### Prediction using DecisionTreeClassifier

It is hard to determine what `max_depth` and `max_leaf_nodes` should be used right away, thus I investigate which `max_leaf_nodes` is the best fit first.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
best_fit={}
best_test={}
for i in range(2,100):
    clf2=DecisionTreeClassifier(max_depth=10,max_leaf_nodes=i)
    clf2.fit(X_train,y_train)
    best_fit[i]=clf2.score(X_train,y_train)
    best_test[i]=clf2.score(X_test,y_test)
max(best_fit.values())

0.7830820770519263

In [ ]:
max(best_test.values())

0.6806020066889632

In [ ]:
pd.Series(data=best_fit, index=range(2,100)).argmax()

97

We found that 97 `max_leaf_nodes` is the best fit, and it is has better prediction score than the `LinearRegression`, but at the same time, it is likely to be overfitted by comparing the difference of the score of train set and test set. 



### Narrowing down the dataset and attemp from differnt approach

#### New train test split and prediciton of LogisticRegression

Since there were too many categories in`Pos`, I narrow down some of the positions for easier prediction by creating a fuction. 

In [ ]:
def position2(x):
    if x in ["CB","S"]:
        return "DB"
    elif x in ["K","P"]:
        return "K"
    elif x in ["LS"]:
        return "OL"
    else:
        return x


In [ ]:
df_com["newPos"]=df_com["Pos"].map(position2)
df_com["newPos"].unique()

array(['DL', 'DB', 'LB', 'WR', 'RB', 'TE', 'OL', 'QB'], dtype=object)

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(df_com[cols],df_com["newPos"],train_size=0.8)

In [ ]:
clf1=LogisticRegression()
clf1.fit(X_train,y_train)
clf1.score(X_train,y_train)

/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.6352596314907872

The prediction score imporved, but it is still not good enough score as a classifier. 

#### Narrowing the predicition column

Also, I narrow the column used for prediction for the possibility that less columns might provide better prediction. 
Weight is assumingly most obvious difference between the position, for example  lines man are probably heavier on average, while height cannot make a clear difference between them.
Thus, I created a list of charts to find which other component is better to give a better predicition  

In [ ]:
import altair as alt
c_list=[]
for i in df_com.columns:
    c=alt.Chart(df_com).mark_circle().encode(
    x=alt.X("Wt",scale=alt.Scale(zero=False)),
    y=alt.Y(i,scale=alt.Scale(zero=False)),
    color="newPos")
    c_list.append(c)

In [ ]:
alt.vconcat(*c_list)

alt.VConcatChart(...)

By looking at the list of charts, except for `Wt`,`Ht`, `newPos`,`Pos`, it appears it has some correlation, thus I decided to use all the columns except for those 4 to make prediciton again.


The below is the example of interactive altair chart with `Wt` and `40yd`. It become more apparent when selecting one data point, so that same position will be presented bigger. 
From that, I found that the same position locates near each other, therefore I decided to make a prediction with two components. 

In [ ]:
sel = alt.selection_single(fields=["newPos"])
c_list[3].encode(
    tooltip=["40yd", "Wt","Vertical","Shuttle"],
    size=alt.condition(sel, alt.value(40),alt.value(10))
    ).add_selection(sel)

alt.Chart(...)

In [ ]:
sub_cols=[i for i in df_com.columns if not i in ["Wt","Ht", "newPos","Pos"]]
sub_cols

['40yd', 'Vertical', 'Bench', 'Broad Jump', '3Cone', 'Shuttle']

In [ ]:
s_list=[]
for i in sub_cols:
    clf=LogisticRegression()
    clf.fit(df_com[["Wt",i]],df_com["newPos"])
    score=clf.score(df_com[["Wt",i]],df_com["newPos"])
    s_list.append(score)
s_list

/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_m

[0.5646349631614199,
 0.5231078365706631,
 0.37173476222371066,
 0.5740120562625586,
 0.5271265907568654,
 0.5328198258539852]

In [ ]:
max(s_list)

0.5740120562625586

It is still not good enough to conclude that these classifier was better. 

#### Prediction of DecisioTreeClassifier with new train test split

In [ ]:
best_fit={}
best_test={}
for i in range(2,100):
    clf2=DecisionTreeClassifier(max_depth=10,max_leaf_nodes=i)
    clf2.fit(X_train,y_train)
    best_fit[i]=clf2.score(X_train,y_train) 
    best_test[i]=clf2.score(X_test,y_test)
max(best_fit.values())

0.8027638190954773

In [ ]:
max(best_test.values())

0.6923076923076923

In [ ]:
pd.Series(data=best_fit, index=range(2,100)).argmax()

97

Although the prediction score imporved, similar to the initial `DecisionTreeClassifier` , it is likely to be overfitted from the difference between the test set and train set. 

### Exploring F-score

In this section, I wanted to search a possiblity for a better prediction through analysis of classification. 
`F-score` was one of the statistical analysis tools for classifier, and by changing the type of averaging process, I want to investigate which one has the higher score as a classification.

This is the explanation of the types of averaging (copied from the source).

'micro':

Calculate metrics globally by counting the total true positives, false negatives and false positives.

'macro':

Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account.

'weighted':

Calculate metrics for each label, and find their average weighted by support (the number of true instances for each label).
This alters ‘macro’ to account for label imbalance; it can result in an F-score that is not between precision and recall.

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_train, clf1.predict(X_train), average='macro')

0.4950097984394265

In [ ]:
f1_score(y_train, clf1.predict(X_train), average='micro')

0.6352596314907872

In [ ]:
f1_score(y_train, clf1.predict(X_train), average='weighted')

0.6142086964365038

In [ ]:
f1_score(y_train, clf2.predict(X_train), average='macro')

0.7746184032955505

In [ ]:
f1_score(y_train, clf2.predict(X_train), average='micro')

0.8027638190954773

In [ ]:
f1_score(y_train, clf2.predict(X_train), average='weighted')

0.798288912352201

In both `LogisticRegression` and `DecisionTreeClassifier`, unweighted averaging, which is the exact same as the regular classifier has the best score.
Thus, there was no better way to imporove the score by changing the averaging. 

## Summary

In conclusion, `DecisionTreeClassifier` with `max_leaf_nodes` of 97 has the best assupmtion to classify the poition of the players in combine. 
However, it is almost convinsing that classifier is overfitting the data, thus it is hard to conclude as a well-constructed classifier.
It can be also concluded that there are weak correlation between the combine data and player's position. 
Or maybe the correlation would become more apparent with more data and narrowing down the categories (`Pos`). 

## References

### Datasets

https://www.kaggle.com/datasets/savvastj/nfl-combine-data

https://www.kaggle.com/datasets/mrframm/nfl-2020-combine

### Source

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html

https://deepai.org/machine-learning-glossary-and-terms/f-score

## Submission

Using the Share & publish link at the top right, enable public sharing in the "Share project" section, and **enable Comment privileges**. Then submit that link on Canvas.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f82393cc-de63-4364-b858-e885e310b2fd' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>